<a href="https://colab.research.google.com/github/MugarraJohnson/Common/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas numpy scipy linalg


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
## Load libraries
import pandas as pd
import numpy as np
from scipy import linalg
import sys
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

From https://www.tensorflow.org/guide/basics

TensorFlow (TF) is an end-to-end platform for machine learning. It supports the following:

1. Multidimensional-array based numeric computation (similar to NumPy.)
2. GPU and distributed processing
3. Automatic differentiation
4. Model construction, training, and export
5. And more

---

Some important points about TF.

* TensorFlow was developed by the Google Brain team for internal use at Google. Then, it was made open source in November 2015.
* Keras is an API (application programming interface) for deep learning calculations. API means that Keras defines a specific interface to write codes.
* Keras was written in Python.
* Keras has multiple backends (libraries that are
responsible for doing the actual calculations): TensorFlow,
CNTK, Theano.
* Keras is focused on easy and fast prototyping, through
user friendliness, modularity, and extensibility.
* Although TF can be used as a backend for Keras, it is
recommended to use tf.keras, which is the implementation
of Keras in TF.


# Data Manipulation
:label:`sec_ndarray`

In order to get anything done,
we need some way to store and manipulate data.
Generally, there are two important things
we need to do with data:
(i) acquire them;
and (ii) process them once they are inside the computer.
There is no point in acquiring data
without some way to store it,
so to start, let's get our hands dirty
with $n$-dimensional arrays,
which we also call *tensors*.
If you already know the NumPy
scientific computing package,
this will be a breeze.
For all modern deep learning frameworks,
the *tensor class* (`ndarray` in MXNet,
`Tensor` in PyTorch and TensorFlow)
resembles NumPy's `ndarray`,
with a few killer features added.
First, the tensor class
supports automatic differentiation.
Second, it leverages GPUs
to accelerate numerical computation,
whereas NumPy only runs on CPUs.
These properties make neural networks
both easy to code and fast to run.



## Getting Started


To start, we import `tensorflow`.
For brevity, practitioners
often assign the alias `tf`.


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

[**A tensor represents a (possibly multidimensional) array of numerical values.**]
In the one-dimensional case, i.e., when only one axis is needed for the data,
a tensor is called a *vector*.
With two axes, a tensor is called a *matrix*.
With $k > 2$ axes, we drop the specialized names
and just refer to the object as a $k^\textrm{th}$-*order tensor*.


TensorFlow provides a variety of functions
for creating new tensors
prepopulated with values.
For example, by invoking `range(n)`,
we can create a vector of evenly spaced values,
starting at 0 (included)
and ending at `n` (not included).
By default, the interval size is $1$.
Unless otherwise specified,
new tensors are stored in main memory
and designated for CPU-based computation.


In [ ]:
x = tf.range(12, dtype = tf.float32)
x

Each of these values is called
an *element* of the tensor.
The tensor `x` contains 12 elements.
We can inspect the total number of elements
in a tensor via the `size` function.


In [ ]:
tf.size(x)

(**We can access a tensor's *shape***)
(the length along each axis)
by inspecting its `shape` attribute.
Because we are dealing with a vector here,
the `shape` contains just a single element
and is identical to the size.


In [ ]:
x.shape

We can [**change the shape of a tensor
without altering its size or values**],
by invoking `reshape`.
For example, we can transform
our vector `x` whose shape is (12,)
to a matrix `X`  with shape (3, 4).
This new tensor retains all elements
but reconfigures them into a matrix.
Notice that the elements of our vector
are laid out one row at a time and thus
`x[3] == X[0, 3]`.


In [ ]:
X = tf.reshape(x, (3, 4))
X

Note that specifying every shape component
to `reshape` is redundant.
Because we already know our tensor's size,
we can work out one component of the shape given the rest.
For example, given a tensor of size $n$
and target shape ($h$, $w$),
we know that $w = n/h$.
To automatically infer one component of the shape,
we can place a `-1` for the shape component
that should be inferred automatically.
In our case, instead of calling `x.reshape(3, 4)`,
we could have equivalently called `x.reshape(-1, 4)` or `x.reshape(3, -1)`.

Practitioners often need to work with tensors
initialized to contain all 0s or 1s.
[**We can construct a tensor with all elements set to 0**] (~~or one~~)
and a shape of (2, 3, 4) via the `zeros` function.


In [ ]:
T = tf.zeros((2, 3, 4))
print(T)
tf.size(T)
tf.shape(T)

Similarly, we can create a tensor
with all 1s by invoking `ones`.


In [ ]:
tf.ones((2, 3, 4))

We often wish to
[**sample each element randomly (and independently)**]
from a given probability distribution.
For example, the parameters of neural networks
are often initialized randomly.
The following snippet creates a tensor
with elements drawn from
a standard Gaussian (normal) distribution
with mean 0 and standard deviation 1.


In [ ]:
W = tf.random.normal(shape=(3, 4))
print(W)

Finally, we can construct tensors by
[**supplying the exact values for each element**]
by supplying (possibly nested) Python list(s)
containing numerical literals.
Here, we construct a matrix with a list of lists,
where the outermost list corresponds to axis 0,
and the inner list corresponds to axis 1.


In [ ]:
T = tf.constant([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
print(T)
#T[0, 0] = 1000 # result in an error message

In [ ]:
# timestamps (0) x patients (1) x features (2)
T = tf.constant([[[74, 128], [79, 116], [71, 116]],
              [[78, 118], [82, 124], [72, 128]],
              [[84, 138], [84, 130], [74, 120]],
              [[82, 126], [76, 156], [82, 132]]])
print(T.shape)
print(T)

In [ ]:
T = tf.transpose(T, perm = [1, 0, 2]) # (0-timestamps, 1-patients, 2-features) -> (1-patients, 0-timestamps, 2-features)
print(T.shape)
print(T)

## Indexing and Slicing

As with  Python lists,
we can access tensor elements
by indexing (starting with 0).
To access an element based on its position
relative to the end of the list,
we can use negative indexing.
Finally, we can access whole ranges of indices
via slicing (e.g., `X[start:stop]`),
where the returned value includes
the first index (`start`) *but not the last* (`stop`).
Finally, when only one index (or slice)
is specified for a $k^\textrm{th}$-order tensor,
it is applied along axis 0.
Thus, in the following code,
[**`[-1]` selects the last row and `[1:3]`
selects the second and third rows**].


In [ ]:
# Axis order: (0-patients, 1-timestamps, 2-features)
print(T)
print('--------')
T[0, -1, :]

`Tensors` in TensorFlow are immutable, and cannot be assigned to.
`Variables` in TensorFlow are mutable containers of state that support
assignments. Keep in mind that gradients in TensorFlow do not flow backwards
through `Variable` assignments.

Beyond assigning a value to the entire `Variable`, we can write elements of a
`Variable` by specifying indices.


In [ ]:
print(X)

In [ ]:
X_var = tf.Variable(X)
X_var[1, 2].assign(6000)
X_var

If we want [**to assign multiple elements the same value,
we apply the indexing on the left-hand side
of the assignment operation.**]
For instance, `[:2, :]`  accesses
the first and second rows,
where `:` takes all the elements along axis 1 (column).
While we discussed indexing for matrices,
this also works for vectors
and for tensors of more than two dimensions.


In [ ]:
print(X)
X_var = tf.Variable(X)
print(X_var)

In [ ]:
X_var = tf.Variable(X)
print(X_var)
X_var[:2, :].assign(tf.ones(X_var[:2,:].shape, dtype=tf.float32) * 12)
X_var

## Operations

Now that we know how to construct tensors
and how to read from and write to their elements,
we can begin to manipulate them
with various mathematical operations.
Among the most useful of these
are the *elementwise* operations.
These apply a standard scalar operation
to each element of a tensor.
For functions that take two tensors as inputs,
elementwise operations apply some standard binary operator
on each pair of corresponding elements.
We can create an elementwise function
from any function that maps
from a scalar to a scalar.

In mathematical notation, we denote such
*unary* scalar operators (taking one input)
by the signature
$f: \mathbb{R} \rightarrow \mathbb{R}$.
This just means that the function maps
from any real number onto some other real number.
Most standard operators, including unary ones like $e^x$, can be applied elementwise.


In [ ]:
print(X)
tf.exp(X)

Likewise, we denote *binary* scalar operators,
which map pairs of real numbers
to a (single) real number
via the signature
$f: \mathbb{R}, \mathbb{R} \rightarrow \mathbb{R}$.
Given any two vectors $\mathbf{u}$
and $\mathbf{v}$ *of the same shape*,
and a binary operator $f$, we can produce a vector
$\mathbf{c} = F(\mathbf{u},\mathbf{v})$
by setting $c_i \gets f(u_i, v_i)$ for all $i$,
where $c_i, u_i$, and $v_i$ are the $i^\textrm{th}$ elements
of vectors $\mathbf{c}, \mathbf{u}$, and $\mathbf{v}$.
Here, we produced the vector-valued
$F: \mathbb{R}^d, \mathbb{R}^d \rightarrow \mathbb{R}^d$
by *lifting* the scalar function
to an elementwise vector operation.
The common standard arithmetic operators
for addition (`+`), subtraction (`-`),
multiplication (`*`), division (`/`),
and exponentiation (`**`)
have all been *lifted* to elementwise operations
for identically-shaped tensors of arbitrary shape.


In [ ]:
x = tf.constant([1.0, 2, 4, 8])
y = tf.constant([2.0, 2, 2, 2])
print(x)
print(y)
x + y, x - y, x * y, x / y, x ** y

In addition to elementwise computations,
we can also perform linear algebraic operations,
such as dot products and matrix multiplications.
We will elaborate on these
in :numref:`sec_linear-algebra`.

We can also [***concatenate* multiple tensors,**]
stacking them end-to-end to form a larger one.
We just need to provide a list of tensors
and tell the system along which axis to concatenate.
The example below shows what happens when we concatenate
two matrices along rows (axis 0)
instead of columns (axis 1).
We can see that the first output's axis-0 length ($6$)
is the sum of the two input tensors' axis-0 lengths ($3 + 3$);
while the second output's axis-1 length ($8$)
is the sum of the two input tensors' axis-1 lengths ($4 + 4$).


In [ ]:
X = tf.reshape(tf.range(12, dtype=tf.float32), (3, 4))
Y = tf.constant([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
print(X)
print(Y)
tf.concat([X, Y], axis=0)
tf.concat([X, Y], axis=1)

Sometimes, we want to
[**construct a binary tensor via *logical statements*.**]
Take `X == Y` as an example.
For each position `i, j`, if `X[i, j]` and `Y[i, j]` are equal,
then the corresponding entry in the result takes value `1`,
otherwise it takes value `0`.


In [ ]:
print(X == Y)

[**Summing all the elements in the tensor**] yields a tensor with only one element.


In [ ]:
print(X)
print(tf.reduce_sum(X))
print(tf.reduce_sum(X, axis = 0))
print(tf.reduce_sum(X, axis = 1))

In [ ]:
# Axis order: (0-patients, 1-timestamps, 2-features)
print(T.shape)
print(T)
tf.math.reduce_mean(T, axis = 0)
tf.math.reduce_mean(T, axis = 1)

## Broadcasting
:label:`subsec_broadcasting`

By now, you know how to perform
elementwise binary operations
on two tensors of the same shape.
Under certain conditions,
even when shapes differ,
we can still [**perform elementwise binary operations
by invoking the *broadcasting mechanism*.**]
Broadcasting works according to
the following two-step procedure:
(i) expand one or both arrays
by copying elements along axes with length 1
so that after this transformation,
the two tensors have the same shape;
(ii) perform an elementwise operation
on the resulting arrays.


In [ ]:
a = tf.reshape(tf.range(3), (3, 1))
b = tf.reshape(tf.range(2), (1, 2))
print(a)
print(b)

Since `a` and `b` are $3\times1$
and $1\times2$ matrices, respectively,
their shapes do not match up.
Broadcasting produces a larger $3\times2$ matrix
by replicating matrix `a` along the columns
and matrix `b` along the rows
before adding them elementwise.


In [ ]:
print(a)
print(b)

In [ ]:
a + b

## Saving Memory

[**Running operations can cause new memory to be
allocated to host results.**]
For example, if we write `Y = X + Y`,
we dereference the tensor that `Y` used to point to
and instead point `Y` at the newly allocated memory.
We can demonstrate this issue with Python's `id()` function,
which gives us the exact address
of the referenced object in memory.
Note that after we run `Y = Y + X`,
`id(Y)` points to a different location.
That is because Python first evaluates `Y + X`,
allocating new memory for the result
and then points `Y` to this new location in memory.


In [ ]:
before = id(Y)
Y = Y + X
id(Y) == before

This might be undesirable for two reasons.
First, we do not want to run around
allocating memory unnecessarily all the time.
In machine learning, we often have
hundreds of megabytes of parameters
and update all of them multiple times per second.
Whenever possible, we want to perform these updates *in place*.
Second, we might point at the
same parameters from multiple variables.
If we do not update in place,
we must be careful to update all of these references,
lest we spring a memory leak
or inadvertently refer to stale parameters.


`Variables` are mutable containers of state in TensorFlow. They provide
a way to store your model parameters.
We can assign the result of an operation
to a `Variable` with `assign`.
To illustrate this concept,
we overwrite the values of `Variable` `Z`
after initializing it, using `zeros_like`,
to have the same shape as `Y`.


In [ ]:
Z = tf.Variable(tf.zeros_like(Y))
print('id(Z):', id(Z))
Z.assign(X + Y)
print('id(Z):', id(Z))

Even once you store state persistently in a `Variable`,
you may want to reduce your memory usage further by avoiding excess
allocations for tensors that are not your model parameters.
Because TensorFlow `Tensors` are immutable
and gradients do not flow through `Variable` assignments,
TensorFlow does not provide an explicit way to run
an individual operation in-place.

However, TensorFlow provides the `tf.function` decorator
to wrap computation inside of a TensorFlow graph
that gets compiled and optimized before running.
This allows TensorFlow to prune unused values,
and to reuse prior allocations that are no longer needed.
This minimizes the memory overhead of TensorFlow computations.


In [ ]:
@tf.function
def computation(X, Y):
    Z = tf.zeros_like(Y)  # This unused value will be pruned out
    A = X + Y  # Allocations will be reused when no longer needed
    B = A + Y
    C = B + Y
    return C + Y

computation(X, Y)

## Conversion to Other Python Objects


[**Converting to a NumPy tensor (`ndarray`)**], or vice versa, is easy.
The converted result does not share memory.
This minor inconvenience is actually quite important:
when you perform operations on the CPU or on GPUs,
you do not want to halt computation, waiting to see
whether the NumPy package of Python
might want to be doing something else
with the same chunk of memory.


In [ ]:
A = X.numpy()
B = tf.constant(A)
type(A), type(B)

To (**convert a size-1 tensor to a Python scalar**),
we can invoke the `item` function or Python's built-in functions.


In [ ]:
a = tf.constant([3.5]).numpy()
a, a.item(), float(a), int(a)

## Summary

The tensor class is the main interface for storing and manipulating data in deep learning libraries.
Tensors provide a variety of functionalities including construction routines; indexing and slicing; basic mathematics operations; broadcasting; memory-efficient assignment; and conversion to and from other Python objects.


## Exercises

1. Run the code in this section. Change the conditional statement `X == Y` to `X < Y` or `X > Y`, and then see what kind of tensor you can get.
1. Replace the two tensors that operate by element in the broadcasting mechanism with other shapes, e.g., 3-dimensional tensors. Is the result the same as expected?


[Discussions](https://discuss.d2l.ai/t/187)


In [ ]:
## Mount Google drive folder if running in Colab
if('google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    DIR = '/content/drive/MyDrive/Colab Notebooks/MAHE/MSIS Coursework/OddSem2023MAHE'
    DATA_DIR = DIR + '/Data/'
else:
    DATA_DIR = 'Data/'

**Problem-1**: we will reconstruct missing values in a real ECG signal. To that end, first load the ECG signal data and make a component plot of the loaded ECG vector by executing the cell below. Note the missing values:

In [ ]:
## Load data
FILE = DATA_DIR + 'ecg_missing.txt'
df_ecg = pd.read_csv(FILE, header = None, names = ['ecgvalues'])
df_ecg['ecgvalues'] = df_ecg['ecgvalues'].astype(float)

## Component plot of the ecg signal
fig, ax = plt.subplots(1, 1, figsize = (6, 4))
component_index = range(0, len(df_ecg['ecgvalues']))
ax.scatter(component_index, df_ecg['ecgvalues'], color = 'black', marker = 'o', s = 5)
ax.set_xlabel('Sample Number')
ax.set_ylabel('Normalized Voltage')
ax.set_title('ECG Signal With Missing Values');